In [57]:
using PyPlot
fig, ax = subplots(figsize=(7,7)) 
#pos = ax.imshow(phi[Int(centery)-radius-10:Int(centery)+radius+10, 1:radius+10], cmap="hot")
pos = ax.imshow(phi, cmap="hot")
fig.colorbar(pos, ax=ax)
#plot(collect(LinRange(0, 1, num_spaces+1)), phi[1, :])
savefig("/Users/arun/Desktop/Plots/phiprofile.png", dpi=300)


In [153]:
using PyPlot
fig, ax = subplots(figsize=(7,7)) 
pos = ax.imshow((gradphi), cmap="hot")
fig.colorbar(pos, ax=ax)
#plot(collect(LinRange(0, 1, num_spaces+1)), phi[1, :])
savefig("/Users/arun/Desktop/Plots/gradphiprofile.png", dpi=300)


In [116]:
using PyPlot
fig, ax = subplots(figsize=(7,7)) 
pos = ax.imshow(grad2phi, cmap="hot")
fig.colorbar(pos, ax=ax)
#plot(collect(LinRange(0, 1, num_spaces+1)), phi[1, :])
savefig("/Users/arun/Desktop/Plots/grad2phiprofile.png", dpi=300)


In [59]:
fig, ax = subplots(figsize=(7,7)) 
pos = ax.imshow(c, cmap="hot")
#plot(collect(LinRange(0, 1, num_spaces+1)), c[1, :])
fig.colorbar(pos, ax=ax)
savefig("/Users/arun/Desktop/Plots/cprofile.png", dpi=300)

In [119]:
fig, ax = subplots(figsize=(7,7)) 
pos = ax.imshow(gradc, cmap="hot")
fig.colorbar(pos, ax=ax)
savefig("/Users/arun/Desktop/Plots/gradcprofile.png", dpi=300)

In [90]:
fig, ax = subplots(figsize=(7,7)) 
pos = ax.imshow(grad2c, cmap="hot")
fig.colorbar(pos, ax=ax)
savefig("/Users/arun/Desktop/Plots/grad2cprofile.png", dpi=300)

In [22]:
fig, ax = subplots(figsize=(7,7)) 
pos = ax.imshow(rho, cmap="hot")
fig.colorbar(pos, ax=ax)
savefig("/Users/arun/Desktop/Plots/rhoprofile.png", dpi=300)

In [92]:
fig, ax = subplots(figsize=(7,7)) 
pos = ax.imshow(gradrho, cmap="hot")
fig.colorbar(pos, ax=ax)
savefig("/Users/arun/Desktop/Plots/gradrhoprofile.png", dpi=300)

In [93]:
fig, ax = subplots(figsize=(7,7)) 
pos = ax.imshow(grad2rho, cmap="hot")
fig.colorbar(pos, ax=ax)
savefig("/Users/arun/Desktop/Plots/grad2rhoprofile.png", dpi=300)

In [122]:
fig, ax = subplots(figsize=(7,7)) 
pos = ax.imshow(dcdt, cmap="hot")
fig.colorbar(pos, ax=ax)
savefig("/Users/arun/Desktop/Plots/dcdtprofile.png", dpi=300)

In [123]:
fig, ax = subplots(figsize=(7,7)) 
pos = ax.imshow(drhodt, cmap="hot")
fig.colorbar(pos, ax=ax)
savefig("/Users/arun/Desktop/Plots/drdtprofile.png", dpi=300)

In [86]:
fig, ax = subplots(figsize=(7,7)) 
pos = ax.imshow(rho)
fig.colorbar(pos, ax=ax)
savefig("/Users/arun/Desktop/Plots/rhoprofile.png", dpi=300)

In [56]:
using LinearAlgebra
using PyPlot
using Dates
fig, ax = subplots(figsize=(7,7)) 

#make bump on grid
num_spaces = 100
grid = zeros(Int8, num_spaces+1, num_spaces+1)
centerx = 1
centery = num_spaces/2 + 1
radius = 10
angles = collect(range(-pi/2, pi/2, num_spaces*2))
ys = @. centery + radius * sin(angles)
xs = @. centerx + radius * cos(angles)
dx = 2 / num_spaces
dt = 10^-5

#parameters
delta = 0.1
epsilon = 0.05
v = 0.1


#initialize
c = ones(Float64, num_spaces+1, num_spaces+1)
rho = zeros(Float64, num_spaces+1, num_spaces+1)
phi = zeros(Float64, num_spaces+1, num_spaces+1)


#fill grid
grid[:, 1] .= 1
c[:, 1] .= 1
###    for i in 1:length(xs)
  #      grid[round(Int, ys[i]), 1:round(Int, xs[i])] .= 1
  #      c[round(Int, ys[i]), 1:round(Int, xs[i])] .= 1
 #   end
#
#right boundary
grid[:, end] .= 1
c[:, end] .= 1

#label special boundary points for gradient evaluation
grid = parseGrid(grid)

gradc = grad(c)
gradrho = grad(rho)
gradphi = grad(phi)
grad2c = grad2(c)
grad2rho = grad2(rho)
grad2phi = grad2(phi)

dcdt = 1
drhodt = 1

for time in 1:20000
    ##enforce boundary conditions
    #solve poissons equation
    A, b = assemblePoisson(num_spaces, rho)
    phi = reshape(A \ b, num_spaces+1, num_spaces+1)
    if time == 1
        plot(collect(LinRange(0, 1, num_spaces+1)), phi[10, :], label="beginning")
    end
    #enforcing no flux condition at boundary for all electrode-boundary cells
    c_copy = copy(c)
    rho_copy = copy(rho)
    for i in 1:num_spaces+1
        for j in 1:num_spaces+1
            if grid[i, j] == 4
                #vertical right
                phipart = phi[i, j+1] - phi[i, j]
                matrix = [1 -phipart; -phipart 1]
                soln = [c_copy[i, j+1]; rho_copy[i, j+1]]
                c[i, j], rho[i, j] = matrix \ soln

            elseif grid[i, j] == 5
                #vertical left
                phipart = phi[i, j-1] - phi[i, j]
                matrix = [1 -phipart; -phipart 1]
                soln = [c_copy[i, j-1]; rho_copy[i, j-1]]
                c[i, j], rho[i, j] = matrix \ soln

            elseif grid[i, j] == 6
                #diagonal down
                phipart = phi[i, j+1] + phi[i+1, j] - 2*phi[i, j]
                matrix = [2 -phipart; -phipart 2]
                soln = [(c_copy[i, j+1] + c_copy[i+1, j]); (rho_copy[i, j+1] + rho_copy[i+1, j])]
                c[i, j], rho[i, j] = matrix \ soln

            elseif grid[i, j] == 7
                #diagonal up
                phipart = phi[i, j+1] + phi[i-1, j] - 2*phi[i, j]
                matrix = [2 -phipart; -phipart 2]
                soln = [(c_copy[i, j+1] + c_copy[i-1, j]); (rho_copy[i, j+1] + rho_copy[i-1, j])]
                c[i, j], rho[i, j] = matrix \ soln

            elseif grid[i, j] == 8
                #horizontal up
                phipart = phi[i-1, j] - phi[i, j]
                matrix = [1 -phipart; -phipart 1]
                soln = [c_copy[i-1, 1]; rho_copy[i-1, j]]
                c[i, j], rho[i, j] = matrix \ soln

            elseif grid[i, j] == 9
                #horizontal down
                phipart = phi[i+1, j] - phi[i, j]
                matrix = [1 -phipart; -phipart 1]
                soln = [c_copy[i+1, 1]; rho_copy[i+1, j]]
                c[i, j], rho[i, j] = matrix \ soln
                
            end
        end
    end

    #update c and rho; boundary is taken care of at the top of the loop
    grad2c = grad2(c)
    grad2rho = grad2(rho)
    grad2phi = grad2(phi)

    dcdt = epsilon .* (grad2c .+ graddotgrad(rho, phi) .+ (rho .* grad2phi))
    drhodt = epsilon .* (grad2rho .+ graddotgrad(c, phi) .+ (c .* grad2phi))
    c .+= (dcdt .* dt)
    rho .+= (drhodt .* dt)
end


plot(collect(LinRange(0, 1, num_spaces+1)), phi[10, :], label="end")
plt.legend()
savefig("/Users/arun/Desktop/Plots/testingphiprofile.png", dpi=300)

In [23]:
using PyPlot
fig, ax = subplots(figsize=(7,7)) 
pos = ax.imshow(grid, cmap="hot")
fig.colorbar(pos, ax=ax)
#plot(collect(LinRange(0, 1, num_spaces+1)), phi[500, :])
savefig("/Users/arun/Desktop/Plots/parsedgrid.png", dpi=300)
